# Functional Programming

## Anonymous Functions

F# provides an alternative way to define functions using the keyword fun; you can see this in the following example.
Ordinarily, you would use this notation when it is not necessary to give a name to a function, so these are referred to as **anonymous functions**
and sometimes called **lambda functions** or even just **lambdas**.

In [54]:
function n -> n * n * n

<fun:it@1-24> : (int -> int)

In [55]:
fun n -> n * n * n

<fun:it@1-25> : (int -> int)

In [56]:
(fun n -> n * n * n) 3

27

In [57]:
let x = (fun x y -> x + y) 1 2
x

3

*Q.* Rewrite `let add x y = x + y` using lambdas. There are two alternatives. Compare the signatures of these alternative functions with the original.

## Type Annotations

F# compiler may occasionally need  more information, which we provide in form of **type annotations**.

In [58]:
let add x y = x + y
add

<fun:it@2-26> : (int -> (int -> int))

In [59]:
let add x (y:float) = x + y
add

<fun:it@2-27> : (double -> (double -> double))

## Type Inference

In the absence of additional type information, the following function is assumed to work with integers:

In [60]:
let squareAndAdd a b = a * a + b
squareAndAdd

<fun:it@2-28> : (int -> (int -> int))

A single type annotation on a is sufficient to indicate that `a * a` is an operation on `float` values and
thus returns a `float` value, and that `a * a + b` is also an operation on `float`:

In [61]:
let squareAndAdd (a:float) b = a * a + b
squareAndAdd

<fun:it@2-29> : (double -> (double -> double))

In general, you can place such annotations on any of the function arguments or directly when you use
them in the body of the function. If you want, you can also give full type annotations for the arguments
and return type of a function:

In [62]:
let squareAndAdd (a:float) (b:float) : float = a * a + b
squareAndAdd

<fun:it@2-30> : (double -> (double -> double))

## Concepts

F# is different in many ways from a standard imperative languages, but there are a few major differences that are particularly important to understand:

- Composition: function-oriented rather than object-oriented
- Expressions rather than statements
- Algebraic types for creating domain models
- Pattern matching for flow of control


## Composition

In [63]:
let double x = x + x
double

<fun:it@2-31> : (int -> int)

In [64]:
let square x = x * x
square

<fun:it@2-32> : (int -> int)

In [65]:
(double >> square)

<fun:it@1-33> : (int -> int)

In [66]:
(double >> square) 3

36

In [67]:
let double_square = double >> square
double_square 3

36

**Building with composition:** Composition is the *glue* that allows us build larger systems from smaller ones. This is not an optional technique, but is at the very heart of the functional style.

In [68]:
List.map square

<fun:it@1-35> : (FSharpList`1 -> FSharpList`1)

In [69]:
// functions as values
let squareclone = square
let result = [1..10] |> List.map squareclone
result

[1; 4; 9; 16; 25; 36; 49; 64; 81; 100]

In [70]:
// functions taking other functions as parameters
let execFunction aFunc aParam = aFunc aParam
let result2 = execFunction square 12
result2

144

## Parameter-less functions

Let's have a function that prints a message

In [71]:
let sayHello = printfn "Hello World!"     // not what we want

Hello World!


In [72]:
sayHello

The fix is to use a lambda.

In [73]:
let sayHello = fun () -> printfn "Hello World!"   // good

sayHello()

Hello World!


or to add a unit parameter to the function

In [74]:
let sayHello() = printfn "Hello World!" // good
sayHello()

Hello World!


## Tuples vs. Multiple Parameters 

In [75]:
// a function that takes two distinct parameters
let addTwoParams x y = x + y
addTwoParams

<fun:it@3-36> : (int -> (int -> int))

In [76]:
// another function that takes a single tuple parameter but looks like it takes two ints
let addTupleParams (x,y) = x + y
addTupleParams

<fun:it@3-37> : (Tuple`2 -> int)

Let’s look at the signatures (it is always a good idea to look at the signatures if you are unsure)

```fsharp
val addTwoParams : int -> int -> int        // two params
val addTupleParams : int * int -> int       // tuple->int
```

Now let’s use them:

In [77]:
addTwoParams 1 2      // ok - uses spaces to separate args

3

In [78]:
addTwoParams (1,2)    // error trying to pass a single tuple

This expression was expected to have type
    'int'    
but here has type
    ''a * 'b'    

In [79]:
addTupleParams (1,2) 

3

See more on
- [function definitions](https://fsharpforfunandprofit.com/posts/defining-functions/)
- [function calls](https://fsharpforfunandprofit.com/posts/convenience-partial-application/)
- [function compositions](https://fsharpforfunandprofit.com/posts/convenience-functions-as-interfaces/)

## Scope

In [80]:
// Split a string into words at spaces
let splitAtSpaces (text:string) =
    text.Split ' '
    |> Array.toList

// Analyze a string for duplicate words
let wordCount text =
    let words = splitAtSpaces text in
    let wordSet = Set.ofList words in
    let numWords = words.Length in
    let numDups = numWords - wordSet.Count in
    (numWords, numDups)
    
wordCount "Analyze a string for duplicate words in a string"

(9, 2)

## Understanding Scope

Local values, such as `words` and `wordCount`, can't be accessed outside their scope.

In the case of variables defined using `let`, the scope of the value is the entire expression that follows the definition, although not the definition itself.

Here are two examples of invalid definitions that try to access variables outside their scope. As you see, let definitions follow a sequential,
top-down order, which helps ensure that programs are well-formed and free from many bugs related to uninitialized values:

In [81]:
let badDefinition1 =    
    let words = splitAtSpaces text
    let text = "We three kings"
    words.Length
badDefinition1

The value or constructor 'text' is not defined. Maybe you want one of the following:
   Text

In [82]:
let badDefinition2 = badDefinition2 + 1

The value or constructor 'badDefinition2' is not defined.

Within function definitions, you can **outscope** values by declaring another value of the same name.

In [83]:
(* This code is equivalent to:
let powerOfFourPlusTwo n =
    let n1 = n * n
    let n2 = n1 * n1
    let n3 = n2 + 2
    n3
*)
let powerOfFourPlusTwo n =
    let n = n * n
    let n = n * n
    let n = n + 2
    n
powerOfFourPlusTwo 3    

83

Because **let** bindings are simply a kind of expression, you can use them in a nested fashion. For example:

In [84]:
let powerOfFourPlusTwoTimesSix n =
    let n3 =
        let n1 = n * n
        let n2 = n1 * n1
        n2 + 2
    let n4 = n3 * 6
    n4
powerOfFourPlusTwoTimesSix 2

108

Here, `n1` and `n2` are values defined locally by let bindings within the expression that defines `n3`.
These local values aren’t available for use outside their scope. For example, this code gives an error:

In [85]:
let invalidFunction n =
    let n3 =
        let n1 = n + n
        let n2 = n1 * n1
        n1 * n2
    let n4 = n1 + n2 + n3 // Error! n3 is in scope, but n1 and n2 are not!
    n4

The value or constructor 'n1' is not defined. Maybe you want one of the following:
   n
   n3
The value or constructor 'n2' is not defined. Maybe you want one of the following:
   n
   n3

Following code that will not compile, because on the third line you change
the value of `x` from an integer to the string "change me", and then on the fourth line you try to
add a string and an integer, which is illegal in F#, so you get a compile error:

In [86]:
let changeType () =
    let x = 1             // bind x to an integer
    let x = "change me"   // rebind x to a string
    let x = x + 1         // attempt to rebind to itself plus an integer
    printfn x

The type 'int' does not match the type 'string'
The type 'int' does not match the type 'string'

## Expressions

In functional languages, there are no statements, only expressions. That is, every chunk of code always returns a value, and larger chunks are created by combining smaller chunks using composition rather than a serialized list of statements.

```c#
// statement-based code in C#
int result;     
if (aBool)
{
  result = 42; 
}
Console.WriteLine("result={0}", result);
```

```c#
// expression-based code in C#
int result = (aBool) ? 42 : 0;
Console.WriteLine("result={0}", result);
```

In [87]:
let aBool = true

// expression-based code in F#
let result =
    if aBool then
        42
    else
        12*0
printfn "result=%i" result 

result=42


In [88]:
printfn "result=%i" (if aBool then 42 else 0) 

result=42


## Recursion

In [1]:
// calculate power (imperative way)
let pow x y = 
    let mutable res = 1
    let mutable count = y
    while count > 0 do
        res <- res * x 
        count <- count - 1
    res
    
pow    

<fun:it@10> : (int -> (int -> int))

In [2]:
pow 2 13

8192

**Recursion** means defining a function in terms of itself; in other words, the function calls itself within its definition. 

- Recursion is often used in functional programming where you would use a loop in imperative programming.

- Many believe that algorithms are much easier to understand when expressed in terms of recursion rather than loops.

- “Makes sense” because calls to same function solve “simpler” problems

In [10]:
let rec pow2 x y = 
    if y = 0
    then 1
    else x * pow2 x (y-1)
        
pow2

<fun:it@6-5> : (int -> (int -> int))

In [9]:
pow2 2 13

8192

Recursion more powerful than loops

- We won’t use a single loop in F#
- Loops often (not always) obscure simple, elegant solutions

In [93]:
let l = [1; 2; 3; 4; 5]
l

[1; 2; 3; 4; 5]

In [94]:
let rec sum_list (xs : int list) = 
    if List.isEmpty xs
    then 0
    else List.head xs + sum_list (List.tail xs)

sum_list

<fun:it@6-40> : (FSharpList`1 -> int)

In [95]:
sum_list [1; 5 ; 10]

16

Because `list` type comes from .NET Framework, it's actually the class `System.List` which have some useful members

In [96]:
let rec sum_list (xs : int list) = 
    if xs.IsEmpty
    then 0
    else xs.Head + sum_list xs.Tail

sum_list

<fun:it@6-41> : (FSharpList`1 -> int)

In [97]:
sum_list [1; 5 ; 10]

16

In [98]:
[1 .. 100]

[1; 2; 3; 4; 5; 6; 7; 8; 9; 10; 11; 12; 13; 14; 15; 16; 17; 18; 19; 20; 21; 22;
 23; 24; 25; 26; 27; 28; 29; 30; 31; 32; 33; 34; 35; 36; 37; 38; 39; 40; 41; 42;
 43; 44; 45; 46; 47; 48; 49; 50; 51; 52; 53; 54; 55; 56; 57; 58; 59; 60; 61; 62;
 63; 64; 65; 66; 67; 68; 69; 70; 71; 72; 73; 74; 75; 76; 77; 78; 79; 80; 81; 82;
 83; 84; 85; 86; 87; 88; 89; 90; 91; 92; 93; 94; 95; 96; 97; 98; 99; 100]

In [99]:
sum_list [1 .. 100]

5050

In [100]:
1 :: []

[1]

In [101]:
1 :: [2; 3]

[1; 2; 3]

In [23]:
let rec countdown (x : int) =
    if x = 0
    then []
    else x :: countdown (x-1)
    
countdown

<fun:it@6-14> : (int -> FSharpList`1)

In [24]:
countdown 10

[10; 9; 8; 7; 6; 5; 4; 3; 2; 1]

### Recursion again

Functions over lists are usually recursive
- Only way to "get to all the elements"
    - What should the answer be for the empty list?
    - What should the answer be for a non-empty list?
- Typically in terms of the answer for the tail of the list!

Similarly, functions that produce lists of potentially any size will be recursive
- You create a list out of smaller lists

In [21]:
let rec append (xs : int list) (ys : int list) =
    if xs.IsEmpty 
    then ys
    else xs.Head :: append xs.Tail ys
    
append

<fun:it@6-13> : (FSharpList`1 -> (FSharpList`1 -> FSharpList`1))

In [25]:
append [1; 2; 3 ; 4] (countdown 3)

[1; 2; 3; 4; 3; 2; 1]

### Avoid repeated recursion

Consider this code and the recursive calls it makes
- Don't worry about calls to `isEmpty`, `head`, and `tail` because they do a small constant amount of work

In [12]:
let rec bad_max (xs : int list) =
    if xs.IsEmpty
    then 0 // horrible style; fix later
    else if xs.Tail.IsEmpty
    then xs.Head 
    else if xs.Head > bad_max xs.Tail
    then xs.Head
    else bad_max xs.Tail

bad_max

<fun:it@10-6> : (FSharpList`1 -> int)

In [11]:
open System.Diagnostics

let timeIt f x = 
    let t = System.Diagnostics.Stopwatch()
    t.Start()
    let res = (f x)
    t.Stop()
    res, t.Elapsed.TotalMilliseconds

In [28]:
[1  .. 25]

[1; 2; 3; 4; 5; 6; 7; 8; 9; 10; 11; 12; 13; 14; 15; 16; 17; 18; 19; 20; 21; 22;
 23; 24; 25]

In [27]:
timeIt bad_max [25 .. -1 .. 1]

(25, 0.0377)

In [30]:
timeIt bad_max [1 .. 28]

(28, 10032.0354)

![](img/bar_rec.png)

### Math never lies

Suppose one bad_max call's `if-then-else logic` and calls to `head`, `isEmpty`, `tail` take $10^{-7}$ seconds

- Then bad_max [50,49,...,1] takes 50 x $10^{-7}$ seconds
- And bad_max [1,2,...,50] takes 1.12 x $10^8$ seconds (over 3.5 years)
    - bad_max [1,2,...,55]takes over 1 century
    - Buying a faster computer won’t help much 
    
The key is not to do repeated work that might do repeated work that might do...
- Saving recursive results in local bindings is essential...

In [19]:
let rec good_max (xs : int list) =
    if xs.IsEmpty
    then 0 // horrible style; fix later
    else if xs.Tail.IsEmpty
    then xs.Head
    else
        let tail_ans = good_max xs.Tail
        if xs.Head > tail_ans
        then xs.Head
        else tail_ans
good_max

<fun:it@11-11> : (FSharpList`1 -> int)

In [20]:
timeIt good_max [1 .. 55]

(55, 0.155)

![](img/good_rec.png)

## Pattern matching for flow of control

Most imperative languages offer a variety of control flow statements for branching and looping:

- if-then-else (and the ternary version bool ? if-true : if-false)
- case or switch statements
- for and foreach loops, with break and continue
- while and until loops
- and even the dreaded goto

F# does support some of these, but F# also supports the most general form of conditional expression, which is **pattern-matching**.

A typical matching expression that replaces if-then-else looks like this:

In [112]:
let booleanExpression = true

match booleanExpression with
| true -> "TRUE" // true branch
| false -> "FALSE" // false branch

"TRUE"

### Patterns

In general the syntax is:

        match e_0 with 
        | p_1 => e_1
        | p_2 => e_2
        ....
        | p_n => e_n

Each pattern is a constructor name followed by the right number of variables (i.e., `C` or `C x` or `C(x,y)` or ...)

- Syntactically most patterns (all today) look like expressions
- But patterns are not expressions
    - We do not evaluate them
    - We see if the result of `e_0` matches them

### Why this way is better

0. You can use pattern-matching to write your own testing and data-extractions functions if you must
1. You cannot forget a case (inexhaustive pattern-match warning)
2. You cannot duplicate a case (a type-checking error)
3. You will not forget to test the variant correctly and get an exception (like List.head [])
4. Pattern-matching can be generalized and made more powerful, leading to elegant and concise code

In [113]:
// This function calculates the nth term in the Fibonacci sequence.
let rec fib x =
    match x with
    | 1 -> 1
    | 2 -> 1
    | x -> fib (x - 1) + fib (x - 2)

fib

<fun:it@8-49> : (int -> int)

In [114]:
fib 11

89

### Lists are datatypes

Do not use `head`, `tail`, or `isEmpty` either
- [] and :: are constructors too
- (strange syntax, particularly infix)

In [115]:
// list summation using recursion and pattern matching
let rec sum_list_pm (xs : int list) =
    match xs with
    | [] -> 0
    | x::xs -> x + sum_list_pm xs
sum_list_pm 

<fun:it@6-50> : (FSharpList`1 -> int)

In [116]:
sum_list_pm [1; 2; 3; 4]

10

In [117]:
let rec append xs ys =
    match xs with
    | [] -> ys
    | x::xs -> x :: append xs ys

append

In [118]:
append (countdown 5) [1 .. 5]

[5; 4; 3; 2; 1; 1; 2; 3; 4; 5]

## Why pattern-matching

- Pattern-matching is better for options and lists for the same reasons as for all datatypes
    - No missing cases, no exceptions for wrong variant, etc.
- We just learned the other way first for pedagogy
     - Do not use `isEmpty`, `Head`, `Tail`
- So why are `isEmpty`, `Head`, etc. predefined?
    - For passing as arguments to other functions
    - Because sometimes they are convenient
    - But not a big deal: could define them yourself

## Algebraic Types

The type system in F# is based on the concept of algebraic types. That is, new compound types are built by combining existing types in two different ways:

- First, a combination of values, each picked from a set of types. These are called “product” types.
- Of, alternately, as a disjoint union representing a choice between a set of types. These are called “sum” types.


In [119]:
//declare it
type IntAndBool = {intPart: int; boolPart: bool}

//use it
let x = {intPart=1; boolPart=false}
x

{intPart = 1;
 boolPart = false;}

In [120]:
//declare it
type IntOrBool = 
  | IntChoice of int
  | BoolChoice of bool

//use it
let y = IntChoice 42
let z = BoolChoice true
y, z

(IntChoice 42, BoolChoice true)